In [5]:
#Biblioteca para localizar a instalação do Spark no sistema e iniciae o Spark
import findspark
findspark.init()
import pyspark
#Gerenciar sessões do Spark
from pyspark.sql import SparkSession
#Funções do Spark SQL
from pyspark.sql.functions import *
#Tipos de dados do Spark SQL,
from pyspark.sql.types import *
from pyspark.sql.functions import to_timestamp


In [6]:
#Criando uma instância do SparkSession
spark = (pyspark.sql.SparkSession.builder \
        .appName("Tratamento") \
        .getOrCreate())

In [9]:
#Lendo o arquivo csv e retonando um objeto DataFrame
df = spark.read.csv("caso.csv", header=True, encoding='ISO-8859-1',inferSchema=True)

+-------------------+-----+----+----------+---------+------+---------------+-------+-------------------------+--------------------+--------------+------------------------------+----------+
|               date|state|city|place_type|confirmed|deaths|order_for_place|is_last|estimated_population_2019|estimated_population|city_ibge_code|confirmed_per_100k_inhabitants|death_rate|
+-------------------+-----+----+----------+---------+------+---------------+-------+-------------------------+--------------------+--------------+------------------------------+----------+
|2022-03-27 00:00:00|   AP|null|     state|   160328|  2122|            734|   true|                   845731|              861773|            16|                   18604.43527|    0.0132|
|2022-03-26 00:00:00|   AP|null|     state|   160321|  2122|            733|  false|                   845731|              861773|            16|                     18603.623|    0.0132|
|2022-03-25 00:00:00|   AP|null|     state|   160314|  

In [27]:
df = df.withColumnRenamed("ID_unico_cliente","ID_UNICO_CLIENTE") \
     .withColumnRenamed("cidade","CIDADE") \
     .withColumnRenamed("estado","ESTADO") \
     .withColumnRenamed("ID_Pedido","ID_PEDIDO") \
     .withColumnRenamed("status_pedido","STATUS_PEDIDO") \
     .withColumnRenamed("ID_item_pedido","ID_ITEM_PEDIDO") \
     .withColumnRenamed("ID_produto","ID_PRODUTO") \
     .withColumnRenamed("preco","PRECO") \
     .withColumnRenamed("frete","FRETE") \
     .withColumnRenamed("categoria_produto","CATEGORIA_PRODUTO") 

In [29]:
# Definindo os tipos das colunas
df = df.withColumn("DATA_PEDIDO", to_timestamp("DATA_PEDIDO", "yyyy-MM-dd HH:mm:ss")) \
       .withColumn("DATA_APROVACAO", to_timestamp("DATA_APROVACAO", "yyyy-MM-dd HH:mm:ss")) \
       .withColumn("DATA_ENTREGA", to_timestamp("DATA_ENTREGA", "yyyy-MM-dd HH:mm:ss")) \
       .withColumn("DATA_ENTREGA_ESTIMADA", to_date("DATA_ENTREGA_ESTIMADA", "yyyy-MM-dd")) \
       .withColumn("QTD_FOTOS_PRODUTO", col("qtd_fotos_produto").cast("integer")) \
       .withColumn("AVALIACAO_COMPRA", col("avaliacao_compra").cast("integer")) 


In [30]:
df.printSchema()

root
 |-- ID_UNICO_CLIENTE: string (nullable = true)
 |-- CIDADE: string (nullable = true)
 |-- ESTADO: string (nullable = true)
 |-- ID_PEDIDO: string (nullable = true)
 |-- STATUS_PEDIDO: string (nullable = true)
 |-- DATA_PEDIDO: timestamp (nullable = true)
 |-- DATA_APROVACAO: timestamp (nullable = true)
 |-- DATA_ENTREGA: timestamp (nullable = true)
 |-- DATA_ENTREGA_ESTIMADA: date (nullable = true)
 |-- ID_ITEM_PEDIDO: integer (nullable = true)
 |-- ID_PRODUTO: string (nullable = true)
 |-- PRECO: double (nullable = true)
 |-- FRETE: double (nullable = true)
 |-- CATEGORIA_PRODUTO: string (nullable = true)
 |-- QTD_FOTOS_PRODUTO: integer (nullable = true)
 |-- AVALIACAO_COMPRA: integer (nullable = true)



In [31]:
df_dim_cliente = df.select("ID_UNICO_CLIENTE","CIDADE","ESTADO")
df_dim_produto = df.select("ID_PRODUTO","CATEGORIA_PRODUTO","QTD_FOTOS_PRODUTO")
df_ft_pedido = df.select("ID_UNICO_CLIENTE","ID_PRODUTO","ID_PEDIDO","STATUS_PEDIDO","DATA_PEDIDO","DATA_APROVACAO","DATA_ENTREGA","DATA_ENTREGA_ESTIMADA","PRECO","FRETE")

In [32]:
df_dim_cliente = df_dim_cliente.dropDuplicates(["ID_UNICO_CLIENTE"])
df_dim_produto = df_dim_produto.dropDuplicates(["ID_PRODUTO"])

In [35]:
#Inserindo DataFrame no BD
df_dim_cliente.write \
    .format("jdbc") \
    .option("url", "jdbc:mysql://localhost:3306/MOSI") \
    .option("driver","com.mysql.cj.jdbc.Driver") \
    .option("dbtable", "DIM_CLIENTE") \
    .option("user", "Diego") \
    .option("password", "iuZ5XZxakXHzBG0e") \
    .mode("append") \
    .save()

In [37]:
#Inserindo DataFrame no BD
df_dim_produto.write \
    .format("jdbc") \
    .option("url", "jdbc:mysql://localhost:3306/MOSI") \
    .option("driver","com.mysql.cj.jdbc.Driver") \
    .option("dbtable", "DIM_PRODUTO") \
    .option("user", "Diego") \
    .option("password", "iuZ5XZxakXHzBG0e") \
    .mode("append") \
    .save()

In [ ]:
#Inserindo DataFrame no BD
df_ft_pedido.write \
    .format("jdbc") \
    .option("url", "jdbc:mysql://localhost:3306/MOSI") \
    .option("driver","com.mysql.cj.jdbc.Driver") \
    .option("dbtable", "FT_PEDIDO") \
    .option("user", "Diego") \
    .option("password", "iuZ5XZxakXHzBG0e") \
    .mode("append") \
    .save()